## Final Project Exploratory Visualization

In this assignment, I will be using data from our final project dataset - Bureau of Labor Statistics (BLS) inflation data - and other government datasets pertaining to earnings, interest rates, unemployment, GDP growth and recessions. The analysis will involve 4 steps that are outlined below: 
1. Acquire, merge and transform various data from BlS, Bureau of Economic Analysis (BED) and the St. Louis Fed.
2. Inspect the data and come up with three (3) hypothesis.
3. Investigate the relevant data to find evidence for or against the hypothesis
4. For only 1 hypotheses --> describe the exploration process, noting changes and refinements that were made.
5. For the other 2 hypotheses --> list each hypothesis and conclusion; provide beginning and ending visulization

Note: this assignment will be graded primarily on how thoroughly you explored the data set with various visualizations. Your first and last visualizations should be quite different.


In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
import altair as alt
from sklearn.linear_model import LinearRegression as LR

## Read in necessary inflation, earning, and unemployment data from BLS

| Topic | Source |
| :- | :- |
| Inflation  | BLS Series ID: CUUR0000SA0 |
| Unemployment | BLS Series ID: LNS14000000 | 
| Earnings | BLS Series ID: CES0500000003 |

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving earnings.csv to earnings.csv
Saving fed_funds.csv to fed_funds.csv
Saving gdp.csv to gdp.csv
Saving inflation.csv to inflation.csv
Saving recession.csv to recession.csv
Saving unemployment.csv to unemployment.csv


In [ ]:
# Import inflation data ()
rows = 11
cols = ["Year", "Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
df_inflation_raw = pd.read_csv('/content/inflation.csv', skiprows = rows, usecols = cols)

# Import unemployment data ()
df_unemploy_raw = pd.read_csv('/content/unemployment.csv', skiprows = rows, usecols = cols)

# Import hourly earnings ()
earning_rows = 12
df_earning_raw = pd.read_csv('/content/earnings.csv', skiprows = earning_rows, usecols = cols)


In [ ]:
def bls_data_transform (df, melt_var = 'Year', new_col = "Inflation"):
    '''transform dataset to change rows to year month and keep
    only months with values'''
    df.columns = df.columns[[0]].tolist() + list(range(1, 13))  # skip 1st col and change remaining to month count
    new_df = df.melt(melt_var)                                  # create new column variable melt_var
    new_df['Date'] = new_df[[melt_var, 'variable']].astype(str).agg('-'.join,1) # join data and create date variable
    
    # Change to date time --> year; month
    new_df['Date'] = pd.to_datetime(new_df['Date']) 
    new_df['Yr_Quarter'] = new_df['Date'].dt.to_period('Q')
    new_df['Date'] = new_df['Date'].dt.to_period('M')
    
    # Drop year and column variables & rename with new column name
    new_df.drop(['Year', 'variable'], axis=1, inplace=True)
    new_df.rename(columns={'value': new_col}, inplace=True)
    
    # Change order of columns, sort and drop rows with no columns
    column_names = ["Date", 'Yr_Quarter', new_col]
    new_df = new_df.reindex(columns=column_names)
    new_df = new_df.sort_values(by = "Date")
    new_df = new_df[new_df[new_col].notna()]
    
    return new_df
    

In [ ]:
# Apply function to the raw dataset
inflation_df = bls_data_transform(df_inflation_raw)
unemploy_df = bls_data_transform(df_unemploy_raw, new_col = "Unemployment")
earning_df = bls_data_transform(df_earning_raw, new_col = "Hourly_earning")

# Make adjustments to inflation datasets for comparability and to calculate y-o-y percentage change in inflation
inflation_df['inf_percent'] = np.round(inflation_df['Inflation'].pct_change(periods = 12) * 100, decimals = 1)
inflation_df['Inflation'] = np.round(inflation_df['Inflation'], decimals = 1)
earning_df['Hourly_earning'] = np.round(earning_df['Hourly_earning'], decimals = 1)

In [ ]:
# Merge two datasets: 1) inflation + unemployment 2) inflation + unemployment + earnings
vis_df_1 = pd.merge(inflation_df, unemploy_df, on=['Date', 'Yr_Quarter'], how='inner')
vis_df_2 = pd.merge(vis_df_1, earning_df, on=['Date', 'Yr_Quarter'], how='inner')

In [ ]:
# Groupby quarters for merging with BEA & St.Louis Fed data
unemploy_qrtr = vis_df_1.groupby('Yr_Quarter', as_index=False)[['inf_percent', 'Unemployment']].mean().round(decimals = 1)
earning_qrtr = vis_df_2.groupby('Yr_Quarter', as_index=False)[['inf_percent', 'Unemployment', 'Hourly_earning']].mean().round(decimals = 1)


## Read in necessary GDP and Interest Rate data from BEA vis St. Louis Fed

In [ ]:
# Import interest data from St. Louis Fed (source: BEA); Rename columns; and reformat Date Column
ff_rows = 10
ff_cols = ["observation_date", "FEDFUNDS"]
df_fedfunds = pd.read_csv('fed_funds.csv', skiprows = ff_rows, usecols = ff_cols)

# Interest rate column and date transformations
df_fedfunds.rename(columns={'observation_date': 'Date', 'FEDFUNDS' : 'interest_rates'}, inplace=True)
df_fedfunds['Date'] = pd.to_datetime(df_fedfunds['Date']) 
df_fedfunds['Yr_Quarter'] = df_fedfunds['Date'].dt.to_period('Q')
df_fedfunds['Date'] = df_fedfunds['Date'].dt.to_period('M')

# GDP data from St. Louis Fed (soucre: BEA); Rename columns; and reformat Date Column
df_gdp = pd.read_csv('gdp.csv')
df_gdp.rename(columns={'DATE': 'Date', 'A191RP1Q027SBEA' : 'GDP'}, inplace=True)

# GDP column and data transformations
df_gdp['Date'] = pd.to_datetime(df_gdp['Date']) 
df_gdp['Yr_Quarter'] = df_gdp['Date'].dt.to_period('Q')
df_gdp['Date'] = df_gdp['Date'].dt.to_period('M')

# Import Recession data from St. Louis Fed; Rename columns; and reformat Date Column
r_rows = 10
r_cols = ["observation_date", "JHDUSRGDPBR"]
df_recession = pd.read_csv('recession.csv', skiprows = r_rows, usecols = r_cols)
df_recession.rename(columns={'observation_date': 'Date', 'JHDUSRGDPBR' : 'Recession'}, inplace=True)

# Recession column and data transformations
df_recession['Date'] = pd.to_datetime(df_recession['Date']) 
df_recession['Yr_Quarter'] = df_recession['Date'].dt.to_period('Q')
df_recession['Date'] = df_recession['Date'].dt.to_period('M')


## Read in necessary inflation, earning, and unemployment data from BLS

| Topic | Source |
| :- | :- |
| Fed Funds Rate  | Federal Reserve of St. Louis |
| GDP (Quarterly) | Federal Reserve of St. Louis | 
| Recession Flag | Federal Reserve of St. Louis |

In [ ]:

# Merge into 1 dataset: 1) GDP + interest_rates + Yr_Quarter
vis_df_temp = pd.merge(df_gdp, df_fedfunds, on=['Date', 'Yr_Quarter'], how='inner')
vis_df_3 = pd.merge(vis_df_temp, df_recession, on=['Date', 'Yr_Quarter'], how='inner')

## Combine BLS and BEA data for other data visualizations

In [ ]:
# Merge into 1 dataset: 1) GDP + interest_rates + Yr_Quarter
vis_df_4 = pd.merge(vis_df_3, unemploy_qrtr, on=['Yr_Quarter'], how='inner')
vis_df_5 = pd.merge(vis_df_3, earning_qrtr, on=['Yr_Quarter'], how='inner')
vis_df_6 = pd.merge(vis_df_1, df_fedfunds, on=['Date', 'Yr_Quarter'], how='inner')
vis_df_7 = pd.merge(vis_df_1, df_gdp, on=['Date', 'Yr_Quarter'], how='inner')

#Update columns
vis4_cols = ["Date", "Yr_Quarter", "Recession", "GDP", "interest_rates", "inf_percent", "Unemployment"]
vis_df_4 = vis_df_4.reindex(columns=vis4_cols) 
vis5_cols = ["Date", "Yr_Quarter", "Recession", "GDP", "interest_rates", "inf_percent", "Unemployment", "Hourly_earning"]
vis_df_5 = vis_df_5.reindex(columns=vis5_cols) 


## Hypothesis
1. Inflation and interest rates are positively correlated across all time periods, except the most recent time period
2. There is a slight positive relationship between inflation and GDP growth
3. Since 2006, growth in inflation has grown faster than hourly earnings



## Hypothesis #1: 
#### Inflation and interest rates are positively correlated across all time periods, except the most recent time period

### Step 1: 
* Test correlation of inflation and interest rates (also, show correlation with unemployment for further exploration)

In [ ]:
# Set-up correlation matrix
corr_matrix = vis_df_6.iloc[:, 3:].corr().round(decimals = 2)
#corr_matrix

# data preparation
pivot_cols = list(corr_matrix.columns)
corr_matrix['cat'] = corr_matrix.index

# actual chart
base = alt.Chart(corr_matrix).transform_fold(pivot_cols).encode(x="cat:N", y='key:N').properties(height = 500, width = 500) 
boxes = base.mark_rect().encode(color=alt.Color("value:Q", scale=alt.Scale(scheme='blues')))
labels = base.mark_text(size = 12, color = "white").encode(text = alt.Text("value:Q", format = "0.2f"))

boxes + labels

alt.LayerChart(...)


### Finding: 
* Although inflation and interest rates are positively correlated at 0.73, unemployment shows no strong correlation to inflation or interest rates (less than 0.13)

### Step 2: 
* See if there are any apparent trends via a line graph

In [ ]:
new_df = vis_df_6.filter(['Date','inf_percent','interest_rates'], axis=1)
new_df['Date'] = new_df['Date'].astype(str)
new_df['Date'] = pd.to_datetime(new_df['Date'])
new_df.rename(columns={'inf_percent' : 'Inflation', 'interest_rates' : "Interest Rates"}, inplace=True)
new_df.head()


,Date,Inflation,Interest Rates
0,1954-07-01,0.4,0.80
1,1954-08-01,0.0,1.22
2,1954-09-01,-0.4,1.07
3,1954-10-01,-0.7,0.85
4,1954-11-01,-0.4,0.83


In [ ]:
scale = alt.Scale(domain=['Inflation', 'Interest Rates'], range=['	#1874CD', '#CD2626'])

alt.Chart(...)

## Finding:
* It appears that inflation and interest rates have been strongly correlated but there are periods where the correlation does not appear as strong and there are some periods (1990s and after 2008) where there appears to be no correlation at all or there is a lagged response. 

### Step 3: 
* Split the data into 7 different time periods to see if the correlation holds

In [ ]:
# The data represents around 70 years of data so split the time series into 7 periods

period_labels = ['1954 - 1964', '1964 - 1973', '1973 - 1983', '1983 - 1993', '1993 - 2002', '2002 - 2012', '2012 - 2022']

new_df['period'] = pd.cut(new_df['Date'],
                           bins = 7,
                           labels = period_labels)


# Count number of observations in each time period
period_list = list(range(1, new_df.period.str.count("1954 - 1964").sum()+1))
period_list.extend(list(range(1, new_df.period.str.count("1964 - 1973").sum()+1)))
period_list.extend(list(range(1, new_df.period.str.count("1973 - 1983").sum()+1)))
period_list.extend(list(range(1, new_df.period.str.count("1983 - 1993").sum()+1)))
period_list.extend(list(range(1, new_df.period.str.count("1993 - 2002").sum()+1)))
period_list.extend(list(range(1, new_df.period.str.count("2002 - 2012").sum()+1)))
period_list.extend(list(range(1, new_df.period.str.count("2012 - 2022").sum()+1)))

# Observation Count
new_df['Observation'] = period_list


new_df.head()


,Date,Inflation,Interest Rates,period,Observation
0,1954-07-01,0.4,0.80,1954 - 1964,1
1,1954-08-01,0.0,1.22,1954 - 1964,2
2,1954-09-01,-0.4,1.07,1954 - 1964,3
3,1954-10-01,-0.7,0.85,1954 - 1964,4
4,1954-11-01,-0.4,0.83,1954 - 1964,5


In [ ]:
alt.Chart(new_df).transform_fold(
    ['period', 'Inflation','Interest Rates']
).mark_line().encode(
    x = alt.X('Observation:O', title = "Observation", axis=alt.Axis(labels=False)),
    y = alt.Y('value:Q', title = 'Percent Change (y-o-y)'),
    color = alt.Color('key:N', scale = scale, legend = alt.Legend(title = "Metric")),
    tooltip = ["Inflation", "Interest Rates"]
).properties(title='', height = 240, width = 380).facet(facet = 'period:N', columns = 4).resolve_scale()

alt.FacetChart(...)

## Finding:
* When separated out by different time periods, it does appear that interest rates and inflation moved together from 1954 to 1993. However, between 1993 and 2022, this trend started to lessen. 

### Step 4: 
* Run correlation matrix for different time periods to compare.

In [ ]:
# Create new dataframe with correlations spread over different periods
df_corr = pd.DataFrame(new_df.groupby('period')[['Inflation','Interest Rates']].corr().unstack().iloc[:,1])

# Flatten out column hierarchy and rename columns: 
df_corr.columns = ['_'.join(cols).lower()
                     for cols in df_corr.columns.to_flat_index()]
df_corr.rename(columns={'inflation_interest rates': 'Correlation'}, inplace=True)
df_corr = df_corr.reset_index()

# Change Correlation to two decimal places
df_corr['Correlation'] = np.round(df_corr['Correlation'], decimals = 3)


In [ ]:
# show graph
bars = alt.Chart(data = df_corr).mark_bar().encode(
    x= alt.X('Correlation', axis = alt.Axis(title='Correlation (0 - 1)', grid=False)),
    y= alt.Y('period', axis = alt.Axis(title = 'Time Period (n = 7)', grid = False))
    )

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=4 
).encode(
    text='Correlation:Q'
)

(bars + text).properties(title=["Correlation between Inflation and Interest Rates by Time Period", ""], height=500, width = 600).configure_view(
    strokeWidth=0
)

alt.LayerChart(...)

## Final Finding for Hypothesis #1:
* Based on the evidence provided above, it is clear that inflation and interest rates are positively correlated across many time periods but obviously not over the past 10 years (2012 - 2022). Therefore, I would say there is evidence to suggest that my hypotethesis is correct. There are, however, two key limitations to this brief analysis. First, I selected arbitrary time periods for analysis based on the start and end dates of the data. Other selected time periods may have yielded different results. Second, this data does not account for other monetary policies that may have been used by policy makers, such as quantitative easing. 

## Hypothesis #2: 
#### There is a slight positive relationship between inflation and GDP growth

In [ ]:
hyp_2_df = vis_df_7.filter(['Date','inf_percent','GDP'], axis=1)
hyp_2_df['Date'] = hyp_2_df['Date'].astype(str)
hyp_2_df['Date'] = pd.to_datetime(hyp_2_df['Date'])
hyp_2_df.rename(columns={'inf_percent' : 'Inflation'}, inplace=True)
hyp_2_df.head()


,Date,Inflation,GDP
0,1950-01-01,-2.1,16.0
1,1950-04-01,-1.3,14.3
2,1950-07-01,1.7,26.8
3,1950-10-01,3.8,16.2
4,1951-01-01,8.1,21.6


In [ ]:
scale = alt.Scale(domain=['Inflation', 'GDP'], range=['	#1874CD', '#228B22'])

alt.Chart(hyp_2_df).transform_fold(
    ['Inflation','GDP']
).mark_line().encode(
    x = alt.X('yearmonthdate(Date):O', title = "Date", axis=alt.Axis(tickSize=0)),
    y = alt.Y('value:Q', title = 'Percent Change (y-o-y)'),
    color = alt.Color('key:N', scale = scale, legend = alt.Legend(title = "Metric")),
    tooltip = ["Inflation", "GDP"]
).properties(title='Percentage Change in Inflation and GDP (1950 - 2022)', height = 550, width = 1600).interactive().configure_axisX(labelAngle=-25)


alt.Chart(...)

In [ ]:
# control for periods of extreme inflation anything greater than 3% (the fed target rate is 2)
hyp_2_df['High Inflation Period'] = (hyp_2_df['Inflation']>=3).astype(int)

hyp_2_df['GDP Extreme'] = (hyp_2_df['GDP']>=5) | (hyp_2_df['GDP']<=-5)
hyp_2_df['GDP Extreme'] = hyp_2_df['GDP Extreme'].astype(int)

hyp_2_df['Extremes'] = hyp_2_df['High Inflation Period'] + hyp_2_df['GDP Extreme']


In [ ]:
# Come up with scenario analysis to include periods with high inflation; periods with extreme GDP values; periods with either no extreme values or periods with only extreme values
high_inflation = hyp_2_df[hyp_2_df['High Inflation Period'] == 1]
high_gdp = hyp_2_df[hyp_2_df['GDP Extreme'] == 1]

high_either = hyp_2_df[hyp_2_df['Extremes'] == 2]
low_either = hyp_2_df[hyp_2_df['Extremes'] != 2]

low_either.head()

,Date,Inflation,GDP,High Inflation Period,GDP Extreme,Extremes
0,1950-01-01,-2.1,16.0,0,1,1
1,1950-04-01,-1.3,14.3,0,1,1
2,1950-07-01,1.7,26.8,0,1,1
8,1952-01-01,4.3,4.2,1,0,1
9,1952-04-01,2.3,1.4,0,0,0


In [ ]:
# making the scatter on Inflation and GDP
fig = alt.Chart(hyp_2_df, title = "Inflation Vs GDP (all)").mark_point().encode(
  x='Inflation',
  y=alt.Y('GDP', scale=alt.Scale(domain=[-40, 40])))
  
# making the regression line using transform_regression
regression = fig.transform_regression('Inflation','GDP').mark_line(color = "#CD0000")
  
plot_1 = (fig + regression)

# making the scatter on Inflation and GDP
fig_2 = alt.Chart(high_inflation, title = "Inflation Vs GDP (Inflation > 3)").mark_point().encode(
  x='Inflation',
  y=alt.Y('GDP', scale=alt.Scale(domain=[-40, 40])))
  
# making the regression line using transform_regression
regression_2 = fig.transform_regression('Inflation','GDP').mark_line(color = "#CD0000")
  
plot_2 = (fig_2 + regression_2)

# making the scatter on Inflation and GDP
fig_3 = alt.Chart(high_gdp, title = "Inflation Vs GDP (GDP > 5 & GDP < -5)").mark_point().encode(
  x='Inflation',
  y=alt.Y('GDP', scale=alt.Scale(domain=[-40, 40])))
  
# making the regression line using transform_regression
regression_3 = fig.transform_regression('Inflation','GDP').mark_line(color = "#CD0000")
  
plot_3 = (fig_3 + regression_3)

# making the scatter on Inflation and GDP
fig_4 = alt.Chart(high_either, title = "Inflation Vs GDP (High Scenario)").mark_point().encode(
  x='Inflation',
  y=alt.Y('GDP', scale=alt.Scale(domain=[-40, 40])))
  
# making the regression line using transform_regression
regression_4 = fig.transform_regression('Inflation','GDP').mark_line(color = "#CD0000")
  
plot_4 = (fig_4 + regression_4)

# making the scatter on Inflation and GDP
fig_5 = alt.Chart(low_either, title = "Inflation Vs GDP (Low Scenario)").mark_point().encode(
  x='Inflation',
  y=alt.Y('GDP', scale=alt.Scale(domain=[-40, 40])))
  
# making the regression line using transform_regression
regression_5 = fig.transform_regression('Inflation','GDP').mark_line(color = "#CD0000")
  
plot_5 = (fig_5 + regression_5)

plot_1 | plot_2 | plot_3 | plot_4 | plot_5



alt.HConcatChart(...)

## Final Finding for Hypothesis #2:
* Based on the evidence provided above, the findings show that inflation and GDP are slightly positively correlated even when controlling for extreme values at either or both the GDP and inflation levels.  This suggests that many other factors are impacting GDP movements, not just inflation.  

## Hypothesis #3: 
#### Since 2006, growth in inflation has grown faster than hourly earnings

In [ ]:
hyp_3_df = vis_df_2.filter(['Date','inf_percent','Hourly_earning'], axis=1)
hyp_3_df['Date'] = hyp_3_df['Date'].astype(str)
hyp_3_df['Date'] = pd.to_datetime(hyp_3_df['Date'])
hyp_3_df.rename(columns={'inf_percent' : 'Inflation', 'Hourly_earning' : "Hourly Earning"}, inplace=True)

In [ ]:
# Check to see if inflation and hourly earnings are at all correlated
# making the scatter on Inflation and 
fig = alt.Chart(hyp_3_df, title = "Inflation Vs Hourly Earnings").mark_point().encode(
  x=alt.X('Inflation', title = "Inflation (percent change)"),
  y=alt.Y('Hourly Earning', title = "Hourly Earning(USD)")
)
  
# making the regression line using transform_regression
regression = fig.transform_regression('Inflation','Hourly Earning').mark_line(color = "#CD0000")
  
plot = (fig + regression)

plot

alt.LayerChart(...)

In [ ]:
# Get percentage change for earnings; drop na
hyp_3_df['Earnings percent'] = np.round(hyp_3_df['Hourly Earning'].pct_change(periods = 12) * 100, decimals = 1)
hyp_3_df = hyp_3_df[hyp_3_df["Earnings percent"].notna()]
hyp_3_df['Difference'] = hyp_3_df["Earnings percent"] - hyp_3_df["Inflation"] 

# Get the percentage change across all dates
hyp_3_df['Difference Sum'] = hyp_3_df.Difference.mean()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [ ]:
# 1st Graphy

base = alt.Chart(hyp_3_df, title = "Earnings and Inflation: Percentage Difference").encode(
    x=alt.X('Date:O', axis=alt.Axis(tickSize=0, labels=False))
)
bar = base.mark_bar().encode(
    y=alt.Y('Difference:Q', title = "Earnings Growth - Inflation Growth (% Diff)"),
    color=alt.value('gray')
    ).interactive()
line =  base.mark_line(color='red').encode(
    y=alt.Y('Difference Sum:Q')
).interactive()

plot_1 = (bar + line).properties(
    height=450,
    width=700
)

scale = alt.Scale(domain=['Inflation', 'Earnings percent'], range=['	#1874CD', 'violet'])

plot_2 = alt.Chart(hyp_3_df).transform_fold(['Inflation','Earnings percent']).mark_line().encode(
      x = alt.X('yearmonthdate(Date):O', title = "Date", axis=alt.Axis(tickSize=0, labels=False)),
      y = alt.Y('value:Q', title = 'Percentage Change (y-o-y)'),
      color = alt.Color('key:N', scale = scale, legend = alt.Legend(title = "Metric")),
      tooltip = ["Inflation", "Earnings percent"]
      ).properties(title='Percentage Change in Inflation and Earnings (2007 - 2022)',
    height=450,
    width=700).interactive()

plot_1 | plot_2

alt.HConcatChart(...)

## Final Finding for Hypothesis #3:
* Based on the evidence provided above, the findings show that growth in hourly earnings have increased slightly faster than growth in inflation from 2007 to 2022.  Therefore, the hypothesis that hourly earnings have not grown as fast as growth in inflation is incorrect.  With that said, inflation growth has outpaced hourly earnings growth for the last 14 months of observations in 2021 and 2022. 

Extra Check on Hypothesis #2

In [ ]:
# Check All Scenarios
print("Correlation Score (ALL): ", hyp_2_df['Inflation'].corr(hyp_2_df['GDP']))
print("Correlation Score (high inflation): ", high_inflation['Inflation'].corr(high_inflation['GDP']))
print("Correlation Score (extreme GDP): ", high_gdp['Inflation'].corr(high_gdp['GDP']))
print("Correlation Score (high scenario): ", high_either['Inflation'].corr(high_either['GDP']))
print("Correlation Score (low scenario): ", low_either['Inflation'].corr(low_either['GDP']))

Correlation Score (ALL):  0.24150420845896922
Correlation Score (high inflation):  0.25310711225338683
Correlation Score (extreme GDP):  0.2097747910564777
Correlation Score (high scenario):  0.35271413085514325
Correlation Score (low scenario):  -0.14517415836396572
